Create the INT8

In [ ]:
import numpy as np
import os

calibration_data_path = "/home/guoy/led_detection/training/yolov8/yolo_val/extra_npy"
files = sorted([os.path.join(calibration_data_path, f) for f in os.listdir(calibration_data_path) if f.endswith(".npy")])

for f in files[:5]:  # 只检查前5个
    data = np.load(f)
    print(f"检查 {f}: shape={data.shape}, dtype={data.dtype}, min={data.min()}, max={data.max()}")


Gao

In [ ]:
import tensorrt as trt
import numpy as np
import os
import pycuda.driver as cuda

onnx_model_path = "RetinaNet_v1.onnx"
trt_engine_path = "RetinaNet_16_int8.trt"

# Create a logger for TensorRT
TRT_LOGGER = trt.Logger(trt.Logger.INFO)

# Create TensorRT builder and network
builder = trt.Builder(TRT_LOGGER)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
parser = trt.OnnxParser(network, TRT_LOGGER)

# Load and parse the ONNX model
with open(onnx_model_path, "rb") as model_file:
    if not parser.parse(model_file.read()):
        print("❌ Failed to parse the ONNX model!")
        for error in range(parser.num_errors):
            print(f"Error {error}: {parser.get_error(error)}")
        exit(1)

# Create a builder config
config = builder.create_builder_config()
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 8 << 30)  # Set 8GB workspace size

# Enable INT8 precision
config.set_flag(trt.BuilderFlag.INT8)  # Enable INT8 precision

# Provide the INT8 calibrator
class Calibrator(trt.IInt8EntropyCalibrator2):
    def __init__(self, calibration_data_path, batch_size=1): #set the batch size
        trt.IInt8EntropyCalibrator2.__init__(self)
        self.batch_size = batch_size
        self.data_path = calibration_data_path
        self.data_files = sorted(os.listdir(calibration_data_path))  
        self.current_index = 0
        self.device_input = None
        self.cache_file = cache_file
        
        # Preloaded calibration data
        self.data = []
        for file in self.data_files:
            file_path = os.path.join(calibration_data_path, file)
            img = np.load(file_path,allow_pickle=True)  # load the  the calibration data （.npy Format）
            self.data.append(img)
        
        self.device_input = cuda.mem_alloc(self.batch_size * self.data[0].nbytes)  

    
    def get_batch(self, names):
        if self.current_index + self.batch_size > len(self.data):
            return None  
        batch = np.ascontiguousarray(self.data[self.current_index:self.current_index + self.batch_size])
        cuda.memcpy_htod(self.device_input, batch)
        self.current_index += self.batch_size
        return [self.device_input]

    def get_batch_size(self):
        return self.batch_size

    def read_calibration_cache(self):
        # 可选：加载已存在的校准缓存
        return None

    def write_calibration_cache(self, cache):
        # 可选：保存校准缓存
        pass

    def get_algorithm(self):
        """ 必须实现此方法，否则会报错 """
        return trt.CalibrationAlgoType.ENTROPY_CALIBRATION_2  # 选择合适的量化算法


# Assuming you have a folder with calibration data in .npy format
calibration_data_folder = "/home/guoy/led_detection/training/yolov8/yolo_val/extra_npy"  # Path to your .npy files

# 正确的方式：直接赋值 int8_calibrator
config.int8_calibrator = Calibrator(calibration_data_folder)

# Set dynamic input and output shape ranges
profile = builder.create_optimization_profile()
profile.set_shape("input", (1, 3, 640, 640), (1, 3, 640, 640), (1, 3, 640, 640))  # Fixed input shape

# Set dynamic shape range for the output tensors
profile.set_shape("scores", (1,), (18,), (300,))  # Minimum 1 detection, maximum 300 detections
profile.set_shape("labels", (1,), (18,), (300,))
profile.set_shape("boxes", (1, 4), (18, 4), (300, 4))

config.add_optimization_profile(profile)

# Build the TensorRT engine
print("🚀 Building TensorRT engine, please wait...")
engine = builder.build_serialized_network(network, config)
if engine is None:
    print("❌ Failed to build the TensorRT engine!")
    exit(1)

# Save the engine to a file

with open(trt_engine_path, "wb") as f:
    f.write(engine)
print(f"✅ TensorRT engine has been saved to {trt_engine_path}") 

[03/14/2025-14:44:46] [TRT] [I] [MemUsageChange] Init CUDA: CPU +14, GPU +0, now: CPU 35, GPU 842 (MiB)
[03/14/2025-14:44:48] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +2175, GPU +418, now: CPU 2365, GPU 1260 (MiB)


TypeError: __init__(self, ...) called with invalid or missing `self` argument